In [274]:
from transformers import AutoTokenizer

model_ckpt = "cl-tohoku/bert-base-japanese-whole-word-masking"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

In [275]:
import torch
from transformers import AutoModelForSequenceClassification
num_labels=3

model = (AutoModelForSequenceClassification
    .from_pretrained(model_ckpt, num_labels=num_labels))

Some weights of the model checkpoint at cl-tohoku/bert-base-japanese-whole-word-masking were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialize

In [276]:
import pandas as pd
import sklearn.preprocessing as sp
df_kokuritu=pd.read_table('univ/kokuritu',header=None)
df_siritu=pd.read_table('univ/siritu',header=None)
df_kouritu=pd.read_table('univ/kouritu',header=None)
df_kokuritu['type']=0
df_siritu['type']=1
df_kouritu['type']=2
df_concat=pd.concat([df_kokuritu,df_siritu,df_kouritu])
df_concat.columns=['name','type']




In [320]:
X=df_concat.name.to_numpy()
y=df_concat.type.to_numpy().reshape(-1,1) 

In [321]:
import numpy as np
from sklearn.model_selection import train_test_split


In [322]:
X_token=tokenizer(X.tolist(),padding=True,truncation=True)['input_ids']

In [323]:
X_train, X_test, y_train, y_test = train_test_split(X_token, y, random_state=0,stratify=y)

In [324]:
X_train=torch.tensor(X_train)
X_test=torch.tensor(X_test)
y_train=torch.tensor(y_train)
y_test=torch.tensor(y_test)

In [325]:
from torch.utils.data import Dataset

class MyDataset(Dataset):
    def __init__(self, X,y):
        self.X=X
        self.y=y
        self.len = len(X)
        
    def __len__(self):
        return self.len
    
    def __getitem__(self, index):
        return self.X[index],self.y[index]

In [326]:
from torch.utils.data import ConcatDataset

In [331]:
y_train=y_train.squeeze(1)
y_test=y_test.squeeze(1)

In [332]:
train=MyDataset(X_train,y_train)
test=MyDataset(X_test,y_test)


In [333]:
train=ConcatDataset([train,test])

In [334]:
batch_size=4

In [335]:
from torch.utils.data import DataLoader

# 学習用Dataloader
train_dataloader = DataLoader(
    train, 
    batch_size=batch_size, 
    shuffle=True,
    drop_last=True,
    pin_memory=True)
test_dataloader = DataLoader(
    test, 
    batch_size=batch_size, 
    shuffle=True,
    drop_last=True,
    pin_memory=True
)

In [336]:
# Get cpu or gpu device for training.
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")


model =model.to(device)
print(model)

Using cpu device
BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(32000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), e

In [337]:
import torch.nn as nn
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [348]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X).logits
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [349]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X).logits
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [350]:
epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 1.234282  [    0/  808]
loss: 0.073050  [  400/  808]
loss: 0.028816  [  800/  808]
Test Error: 
 Accuracy: 96.0%, Avg loss: 0.095899 

Epoch 2
-------------------------------
loss: 0.129451  [    0/  808]
loss: 0.217448  [  400/  808]
loss: 0.031020  [  800/  808]
Test Error: 
 Accuracy: 97.5%, Avg loss: 0.071629 

Epoch 3
-------------------------------
loss: 0.143803  [    0/  808]
loss: 0.019404  [  400/  808]
loss: 0.077069  [  800/  808]
Test Error: 
 Accuracy: 96.0%, Avg loss: 0.116940 

Epoch 4
-------------------------------
loss: 0.304722  [    0/  808]
loss: 0.063795  [  400/  808]
loss: 0.038304  [  800/  808]
Test Error: 
 Accuracy: 97.0%, Avg loss: 0.063986 

Epoch 5
-------------------------------
loss: 0.360720  [    0/  808]
loss: 0.011751  [  400/  808]
loss: 0.019727  [  800/  808]
Test Error: 
 Accuracy: 98.5%, Avg loss: 0.037370 

Done!


In [370]:
model10=torch.load('10.pt')

In [440]:
name='九州東京大学'
with torch.no_grad():
    print(tokenizer.tokenize(name))
    out=(model10(torch.tensor(tokenizer(name)['input_ids']).unsqueeze(0)).logits)
    out[0][1]=0.01*out[0][1]
    out=nn.Softmax(dim=-1)(out)
ans_dict=['国立','私立','公立']
ans_dict[np.argmax(out)]

['九州', '東京大学']


'国立'

In [422]:
def test_(dataloader, model, loss_fn,alpha=0.1):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X).logits
            pred=nn.Softmax(dim=-1)(pred)
            pred[:,1]=alpha*pred[:,1]
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [432]:
test_(test_dataloader, model, loss_fn,alpha=0.01)

Test Error: 
 Accuracy: 87.1%, Avg loss: 0.983018 

